# Data Cleaning for Deaths Data

This is a revisit of a previous project: https://github.com/Dference/Project-3-Healthcare

In that project, deaths were sorted into all, natural, chronic, and non-chronic categories. This revisit resorts the data to highlight COVID deaths. That being said, no matter how the data is summed up, the numbers never add up to the "All Cause" column in the original dataset. There is no indication in the documentation why this is. Additional, in the original project, much of the calculations were done within the js file. Those calculations will be computed in this file instead.

In [1]:
import pandas as pd
from pathlib import Path
from pprint import pprint
import json

## Cleaning Death Counts Data

In [2]:
# Death Counts data

data = pd.read_csv(Path("Data\Weekly_Provisional_Counts_of_Deaths_by_State_and_Select_Causes__2020-2023_20240922.csv"))
# dropping flag columns and filling NA values with 0
data = data.drop(data.columns[20:], axis=1).fillna(0)

#since we're only looking at annual sums, we can drop these columns
data = data.drop(columns=["Data As Of", "Week Ending Date", "MMWR Week"])

#getting rid of the United States rows for now
data = data[data["Jurisdiction of Occurrence"] != "United States"]

# renaming jurisdiction and year bc i can't type that every time
data = data.rename(columns={"Jurisdiction of Occurrence":"state", "MMWR Year":"year"})

data.head()

,state,year,All Cause,Natural Cause,Septicemia (A40-A41),Malignant neoplasms (C00-C97),Diabetes mellitus (E10-E14),Alzheimer disease (G30),Influenza and pneumonia (J09-J18),Chronic lower respiratory diseases (J40-J47),"Other diseases of respiratory system (J00-J06,J30-J39,J67,J70-J98)","Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)","Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)","Diseases of heart (I00-I09,I11,I13,I20-I51)",Cerebrovascular diseases (I60-I69),"COVID-19 (U071, Multiple Cause of Death)","COVID-19 (U071, Underlying Cause of Death)"
194,Alabama,2020,1082,1009,15.0,196.0,25.0,54.0,28.0,86.0,13.0,25.0,25.0,268.0,81.0,0.0,0.0
195,Alabama,2020,1127,1060,25.0,197.0,17.0,41.0,27.0,72.0,20.0,23.0,34.0,274.0,68.0,0.0,0.0
196,Alabama,2020,1039,976,28.0,201.0,24.0,53.0,31.0,73.0,21.0,23.0,24.0,239.0,45.0,0.0,0.0
197,Alabama,2020,1056,982,16.0,190.0,23.0,54.0,29.0,61.0,11.0,14.0,23.0,284.0,70.0,0.0,0.0
198,Alabama,2020,1026,949,15.0,157.0,16.0,58.0,31.0,76.0,17.0,20.0,23.0,265.0,55.0,0.0,0.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10282 entries, 194 to 10475
Data columns (total 17 columns):
 #   Column                                                                                             Non-Null Count  Dtype  
---  ------                                                                                             --------------  -----  
 0   state                                                                                              10282 non-null  object 
 1   year                                                                                               10282 non-null  int64  
 2   All Cause                                                                                          10282 non-null  int64  
 3   Natural Cause                                                                                      10282 non-null  int64  
 4   Septicemia (A40-A41)                                                                               10282 non-null  float6

In [4]:
# no matter how you sum the numbers, they don't add up to all cause? there is no indiciation in the documentation why this is
# data["sum"] = data.iloc[:,3:17].sum(axis=1)
# data.drop(data.columns[3:17], axis=1)
# data["sum"] = data.iloc[:,4:17].sum(axis=1)
# data.drop(data.columns[4:17], axis=1)

In [5]:
# creating a column for all other causes, leaving us w all cause, nat cause, covid-19 multiple, covid-19 underlying, and all other
data["All Other Cause"] = data.iloc[:,4:15].sum(axis=1)
data = data.drop(data.columns[4:15], axis=1)
data.head()

,state,year,All Cause,Natural Cause,"COVID-19 (U071, Multiple Cause of Death)","COVID-19 (U071, Underlying Cause of Death)",All Other Cause
194,Alabama,2020,1082,1009,0.0,0.0,816.0
195,Alabama,2020,1127,1060,0.0,0.0,798.0
196,Alabama,2020,1039,976,0.0,0.0,762.0
197,Alabama,2020,1056,982,0.0,0.0,775.0
198,Alabama,2020,1026,949,0.0,0.0,733.0


In [6]:
df = data.groupby(["year","state"]).agg("sum")
df

All Cause  Natural Cause  \
year state                                     
2020 Alabama            64354          59772   
     Alaska              5076           4283   
     Arizona            78257          70296   
     Arkansas           38399          35509   
     California        326702         300014   
...                       ...            ...   
2023 Virginia           53647          49641   
     Washington         45514          40564   
     West Virginia      16992          15377   
     Wisconsin          39648          36289   
     Wyoming             3631           3244   

                    COVID-19 (U071, Multiple Cause of Death)  \
year state                                                     
2020 Alabama                                          6860.0   
     Alaska                                            181.0   
     Arizona                                          9575.0   
     Arkansas                                         4097.0   
     California                                      34968.0   
...                                                      ...   
2023 Virginia                                         1286.0   
     Washington                                       1028.0   
     West Virginia                                     377.0   
     Wisconsin                                         714.0   
     Wyoming                                             0.0   

                    COVID-19 (U071, Underlying Cause of Death)  \
year state                                                       
2020 Alabama                                            6406.0   
     Alaska                                              143.0   
     Arizona                                            8810.0   
     Arkansas                                           3600.0   
     California                                        32687.0   
...                                                        ...   
2023 Virginia                                            771.0   
     Washington                                          608.0   
     West Virginia                                       247.0   
     Wisconsin                                           401.0   
     Wyoming                                               0.0   

                    All Other Cause  
year state                           
2020 Alabama                41688.0  
     Alaska                  1880.0  
     Arizona                45200.0  
     Arkansas               24364.0  
     California            207864.0  
...                             ...  
2023 Virginia               36040.0  
     Washington             28702.0  
     West Virginia          10499.0  
     Wisconsin              26284.0  
     Wyoming                 1466.0  

[212 rows x 5 columns]

In [7]:
df["All Cause"]

year  state        
2020  Alabama           64354
      Alaska             5076
      Arizona           78257
      Arkansas          38399
      California       326702
                        ...  
2023  Virginia          53647
      Washington        45514
      West Virginia     16992
      Wisconsin         39648
      Wyoming            3631
Name: All Cause, Length: 212, dtype: int64

In [8]:
# converting the grouped dataframe into a dictionary to eventually export as a json file

# initializing the dictionary
results = {}

for index, row in df.iterrows():
    nested = results
    # iterating over year and state as indexes, i will have the values of 0 and 1
    for i, key in enumerate(index):
        # if at the last key data gets stored in dictionary
        if i == len(index) - 1:
            nested[key] = row.to_dict()
        else:
            # creating a new dictionary for each key
            if key not in nested:
                nested[key] = {}
            # Move deeper into the nested structure
            nested = nested[key]


pprint(results)

{2020: {'Alabama': {'All Cause': 64354.0,
                    'All Other Cause': 41688.0,
                    'COVID-19 (U071, Multiple Cause of Death)': 6860.0,
                    'COVID-19 (U071, Underlying Cause of Death)': 6406.0,
                    'Natural Cause': 59772.0},
        'Alaska': {'All Cause': 5076.0,
                   'All Other Cause': 1880.0,
                   'COVID-19 (U071, Multiple Cause of Death)': 181.0,
                   'COVID-19 (U071, Underlying Cause of Death)': 143.0,
                   'Natural Cause': 4283.0},
        'Arizona': {'All Cause': 78257.0,
                    'All Other Cause': 45200.0,
                    'COVID-19 (U071, Multiple Cause of Death)': 9575.0,
                    'COVID-19 (U071, Underlying Cause of Death)': 8810.0,
                    'Natural Cause': 70296.0},
        'Arkansas': {'All Cause': 38399.0,
                     'All Other Cause': 24364.0,
                     'COVID-19 (U071, Multiple Cause of Death)': 4097

## State Population Estimates
Adding State Population Estimate data to the dictionary to later calculate proportion of deaths

In [9]:
# state population data
state_pop = pd.read_csv(Path("Data/NST-EST2023-ALLDATA.csv"))
state_pop.head()

,NAME,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023
0,Alabama,5031864,5050380,5073903,5108468
1,Alaska,732964,734923,733276,733406
2,Arizona,7186683,7272487,7365684,7431344
3,Arkansas,3014348,3028443,3046404,3067732
4,California,39503200,39145060,39040616,38965193


In [10]:
#  renaming the columns to match the death counts column names for the for loop later
state_pop = state_pop.rename(columns={"POPESTIMATE2020": 2020,
                          "POPESTIMATE2021": 2021,
                          "POPESTIMATE2022": 2022,
                          "POPESTIMATE2023": 2023})

In [11]:
for year in results:
    for state in results[year]:
        # pulling the population estimate for every year and state from the state dataframe
            # have put it in a list so the index will not be added when appending the json, there may be a simpler solution
        pop_est = list(state_pop[state_pop['NAME'] == state][year])
        results[year][state]["pop_est"] = pop_est[0]

results
        

{2020: {'Alabama': {'All Cause': 64354.0,
   'Natural Cause': 59772.0,
   'COVID-19 (U071, Multiple Cause of Death)': 6860.0,
   'COVID-19 (U071, Underlying Cause of Death)': 6406.0,
   'All Other Cause': 41688.0,
   'pop_est': 5031864},
  'Alaska': {'All Cause': 5076.0,
   'Natural Cause': 4283.0,
   'COVID-19 (U071, Multiple Cause of Death)': 181.0,
   'COVID-19 (U071, Underlying Cause of Death)': 143.0,
   'All Other Cause': 1880.0,
   'pop_est': 732964},
  'Arizona': {'All Cause': 78257.0,
   'Natural Cause': 70296.0,
   'COVID-19 (U071, Multiple Cause of Death)': 9575.0,
   'COVID-19 (U071, Underlying Cause of Death)': 8810.0,
   'All Other Cause': 45200.0,
   'pop_est': 7186683},
  'Arkansas': {'All Cause': 38399.0,
   'Natural Cause': 35509.0,
   'COVID-19 (U071, Multiple Cause of Death)': 4097.0,
   'COVID-19 (U071, Underlying Cause of Death)': 3600.0,
   'All Other Cause': 24364.0,
   'pop_est': 3014348},
  'California': {'All Cause': 326702.0,
   'Natural Cause': 300014.0,
  

## Import State GeoJson
Adding the completed dictionary to the state geojson file for ease of use in the chloropleth map

In [12]:
# Open and read the JSON file containing the geographic data for the states
with open('Data/us-states.json', 'r') as file:
    geo_data = json.load(file)


In [13]:
# adding the quantitative data to each of the states
for feature in geo_data["features"]:
    for year in results:
        for state in results[year]:
            if state == feature["properties"]["name"]:
                feature["properties"][year] = results[year][state]

pprint(geo_data["features"][0])

{'geometry': {'coordinates': [[[-87.359296, 35.00118],
                               [-85.606675, 34.984749],
                               [-85.431413, 34.124869],
                               [-85.184951, 32.859696],
                               [-85.069935, 32.580372],
                               [-84.960397, 32.421541],
                               [-85.004212, 32.322956],
                               [-84.889196, 32.262709],
                               [-85.058981, 32.13674],
                               [-85.053504, 32.01077],
                               [-85.141136, 31.840985],
                               [-85.042551, 31.539753],
                               [-85.113751, 31.27686],
                               [-85.004212, 31.003013],
                               [-85.497137, 30.997536],
                               [-87.600282, 30.997536],
                               [-87.633143, 30.86609],
                               [-87.408589, 30.674397

In [14]:
# exporting the dictionary as a json file
with open('Data/data.json', 'w', encoding='utf-8') as f:
    json.dump(geo_data, f, indent=4)

In [15]:
for key in geo_data["features"][0]["properties"]:
    print(key)

name
2020
2021
2022
2023
